# Feast Basic Customer Transactions Example

This is a minimal example of using Feast. In this example we will
1. Create a synthetic customer feature dataset
2. Register a feature set to represent these features in Feast
3. Ingest these features into Feast
4. Create a feature query and retrieve online feature data
5. Create a feature query and retrieve historical feature data

### 0. Configuration

In [1]:
import os

# Feast Core acts as the central feature registry
FEAST_CORE_URL = os.getenv('FEAST_CORE_URL', 'localhost:6565')

# Feast Online Serving allows for the retrieval of real-time feature data
FEAST_ONLINE_SERVING_URL = os.getenv('FEAST_ONLINE_SERVING_URL', 'localhost:6566')

# Feast Batch Serving allows for the retrieval of historical feature data
FEAST_BATCH_SERVING_URL = os.getenv('FEAST_BATCH_SERVING_URL', 'localhost:6567')

### 1. Install Feast SDK

Install from PyPi

In [2]:
!pip install feast

### 2. Import necessary modules

In [3]:
import pandas as pd
import numpy as np
from pytz import timezone, utc
from feast import Client, FeatureSet, Entity, ValueType
from feast.serving.ServingService_pb2 import GetOnlineFeaturesRequest
from feast.types.Value_pb2 import Value as Value
from google.protobuf.duration_pb2 import Duration
from datetime import datetime, timedelta
from random import randrange
import random

### 3. Configure Feast services and connect the Feast client

Connect to Feast Core and Feast Online Serving

In [4]:
client = Client(core_url=FEAST_CORE_URL, serving_url=FEAST_ONLINE_SERVING_URL)

### 4. Create customer features

Here we will create customer features for 5 customers over a month. Each customer will have a set of features for every day.

In [5]:
days = [datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0).replace(tzinfo=utc) \
        - timedelta(day) for day in range(3)][::-1]

customers = [1001, 1002, 1003, 1004, 1005]

In [6]:
customer_features = pd.DataFrame(
    {
        "datetime": [day for day in days for customer in customers],
        "customer_id": [customer for day in days for customer in customers],
        "daily_transactions": [np.random.rand() * 10 for _ in range(len(days) * len(customers))],
        "total_transactions": [np.random.randint(100) for _ in range(len(days) * len(customers))],
    }
)

customer_features.head(10)

,datetime,customer_id,daily_transactions,total_transactions
0,2020-05-25 00:00:00+00:00,1001,8.704802,77
1,2020-05-25 00:00:00+00:00,1002,7.163887,31
2,2020-05-25 00:00:00+00:00,1003,9.935976,68
3,2020-05-25 00:00:00+00:00,1004,1.107980,78
4,2020-05-25 00:00:00+00:00,1005,8.307381,36
5,2020-05-26 00:00:00+00:00,1001,2.416811,14
6,2020-05-26 00:00:00+00:00,1002,4.817735,9
7,2020-05-26 00:00:00+00:00,1003,4.409714,95
8,2020-05-26 00:00:00+00:00,1004,6.617317,6
9,2020-05-26 00:00:00+00:00,1005,1.032525,86


### 5. Create feature set for customer features

Now we will create a feature set for these features. Feature sets are essentially a schema that represent
feature values. Feature sets allow Feast to both identify feature values and their structure. The following feature set contains no features yet.

In [7]:
customer_fs = FeatureSet(
    "customer_transactions",
    entities=[Entity(name='customer_id', dtype=ValueType.INT64)],
    max_age=Duration(seconds=432000)    
)

Here we are automatically inferring the schema from the provided dataset. The two features from the dataset will be added to the feature set

In [8]:
customer_fs.infer_fields_from_df(customer_features, replace_existing_features=True)

Feature daily_transactions (ValueType.DOUBLE) added from dataframe.
Feature total_transactions (ValueType.INT64) added from dataframe.



### 6. Register feature set with Feast Core

The apply() method will register the provided feature set with Feast core, allowing users to retrieve features from this feature set

In [9]:
client.apply(customer_fs)
customer_fs = client.get_feature_set("customer_transactions")
print(customer_fs)

Feature set created: "customer_transactions"
{
  "spec": {
    "name": "customer_transactions",
    "entities": [
      {
        "name": "customer_id",
        "valueType": "INT64"
      }
    ],
    "features": [
      {
        "name": "daily_transactions",
        "valueType": "DOUBLE"
      },
      {
        "name": "total_transactions",
        "valueType": "INT64"
      }
    ],
    "maxAge": "432000s",
    "source": {
      "type": "KAFKA",
      "kafkaSourceConfig": {
        "bootstrapServers": "kafka:9092,localhost:9094",
        "topic": "feast-features"
      }
    },
    "project": "default"
  },
  "meta": {
    "createdTimestamp": "2020-05-27T03:58:07Z",
    "status": "STATUS_PENDING"
  }
}


### 7. Ingest data into Feast for a feature set

In [10]:
client.ingest("customer_transactions", customer_features)

Waiting for feature set to be ready for ingestion...


100%|██████████| 15/15 [00:01<00:00, 13.99rows/s]

Ingestion complete!

Ingestion statistics:
Success: 15/15
Removing temporary file(s)...


'3b988d56-6885-36c6-804e-73ea76b7eae6'

### 8. Retrieve online features

The process of retrieving features from the online API is very similar to that of the batch API. The only major difference is that users do not have to provide timestamps (only the latest features are returned, as long as they are within the maximum age window)

The example below retrieves online features for a single customer: "1001". It is possible to retrieve any features from feast, even outside of the current project.

In [11]:
online_features = client.get_online_features(
    feature_refs=[
        f"daily_transactions",
        f"total_transactions",
    ],
    entity_rows=[
        GetOnlineFeaturesRequest.EntityRow(
            fields={
                "customer_id": Value(
                    int64_val=1001)
            }
        )
    ],
)
print(online_features)

field_values {
  fields {
    key: "customer_id"
    value {
      int64_val: 1001
    }
  }
  fields {
    key: "daily_transactions"
    value {
      double_val: 2.460333315469021
    }
  }
  fields {
    key: "total_transactions"
    value {
      int64_val: 11
    }
  }
}



### The following section requires Google Cloud Platform (Google Cloud Storage and BigQuery)

### 9. Create a batch retrieval query

In order to retrieve historical feature data, the user must provide an entity_rows dataframe. This dataframe contains a combination of timestamps and entities. In this case, the user must provide both customer_ids and timestamps. 

We will randomly generate timestamps over the last 30 days, and assign customer_ids to them. When these entity rows are sent to the Feast Serving API to retrieve feature values, along with a list of feature ids, Feast is then able to attach the correct feature values to each entity row. 

In [19]:
entity_rows = pd.DataFrame(
    {
        "datetime": [day for day in days for customer in customers],
        "customer_id": [customer for day in days for customer in customers],
    }
)

entity_rows.head(10)

,datetime,customer_id
0,2020-05-25 00:00:00+00:00,1001
1,2020-05-25 00:00:00+00:00,1002
2,2020-05-25 00:00:00+00:00,1003
3,2020-05-25 00:00:00+00:00,1004
4,2020-05-25 00:00:00+00:00,1005
5,2020-05-26 00:00:00+00:00,1001
6,2020-05-26 00:00:00+00:00,1002
7,2020-05-26 00:00:00+00:00,1003
8,2020-05-26 00:00:00+00:00,1004
9,2020-05-26 00:00:00+00:00,1005


### 10. Retrieve historical/batch features

Next we will create a new client object, but this time we will configure it to connect to the Batch Serving Service. This service will allow us to retrieve historical feature data.

In [20]:
batch_client = Client(core_url=FEAST_CORE_URL, serving_url=FEAST_BATCH_SERVING_URL)

By calling the `get_batch_features` method we are able to retrieve a `job` object for the exporting of feature data. For every entity and timestamp combination in `entity_rows` we will be receiving a row with feature values joined to it.

In [21]:
job = batch_client.get_batch_features(
                            feature_refs=[
                                f"daily_transactions", 
                                f"total_transactions", 
                               ],
                            entity_rows=entity_rows
                         )

Once the job is complete, it is possible to retrieve the exported data (from Google Cloud Storage) and load it into memory as a Pandas Dataframe.

In [22]:
df = job.to_dataframe()
df.head()

,event_timestamp,customer_id,daily_transactions,total_transactions
0,2020-05-26 00:00:00+00:00,1001,2.416811,14
1,2020-05-26 00:00:00+00:00,1004,6.617317,6
2,2020-05-26 00:00:00+00:00,1003,4.409714,95
3,2020-05-26 00:00:00+00:00,1005,1.032525,86
4,2020-05-26 00:00:00+00:00,1002,4.817735,9
